In [ ]:
#Começa aqui após ter os arquivos
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads
from shapely import wkt

#Shapefile baixado em: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html
'''
file_names = ["2023-Q1.parquet", "2023-Q2.parquet", "2023-Q3.parquet", "2023-Q4.parquet"]
def load_and_process_data(file_names):
    dfs = []
    for file_name in file_names:
        print(f'Carregando dados do arquivo: {file_name}')
        df = pd.read_parquet(file_name)
        df['geometry'] = df['tile'].apply(wkt.loads)
        df['quarter'] = file_name.split('-')[1]  # Assumindo que o nome do arquivo contém o trimestre
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True, axis=0)
    return combined_df

df = load_and_process_data(file_names)
print(df)
Ficou muito pesado
'''
file_name = "2023-Q1.parquet"
df = pd.read_parquet(file_name)


df.rename(columns={'tile': 'geometry'}, inplace=True)

df['geometry'] = df['geometry'].apply(wkt.loads)

df_geo = gpd.GeoDataFrame(df, geometry='geometry')
df_geo.crs = "EPSG:4326"

df_geo.head()

In [ ]:
#shapefile_sp = gpd.read_file('SP_Municipios_2022.zip') 
shapefile_sp = gpd.read_file('BR_Municipios_2022.zip') #Tentar rodar com brasil
distritos = shapefile_sp.to_crs(4326)
distritos.head()

nome_arquivo_excel = "distritos.xlsx"
distritos.to_excel(nome_arquivo_excel, index=False)
#print(f"Arquivo '{nome_arquivo_excel}' salvo com sucesso!")

In [ ]:
# Realiza o join espacial
join_sp = gpd.sjoin(df_geo, distritos, how="inner", op='intersects')
join_sp.head()
# O resultado 'join_sp' conterá as velocidades de internet com a informação dos municípios de São Paulo

In [ ]:
# Supondo que join_sp é o DataFrame com seus dados
# Calcular a média de avg_d_kbps por município
media_velocidade_sp = join_sp.groupby('NM_MUN')['avg_d_kbps'].mean()

# Resetar o índice para transformar em DataFrame
media_velocidade_sp = media_velocidade_sp.reset_index()

# Converter a média de kbps para Mbps diretamente na nova coluna
media_velocidade_sp['avg_d_mbps'] = media_velocidade_sp['avg_d_kbps'] / 1000

# Exibir a média em Mbps
print(media_velocidade_sp)

# Nome do arquivo Excel
nome_arquivo_excel = "media_velocidade_por_municipio.xlsx"
media_velocidade_sp.to_excel(nome_arquivo_excel, index=False)

In [ ]:
# Merge para associar cada município à sua média de velocidade de internet
map_data = distritos.merge(media_velocidade_sp, left_on='NM_MUN', right_on='NM_MUN')

In [ ]:
# Cria um mapa centrado em uma localização de São Paulo
m = folium.Map(location=[-23.5505, -46.6333], zoom_start=7)

# Adiciona a camada Choropleth
folium.Choropleth(
    geo_data=map_data.__geo_interface__,  # Usa a interface GeoJSON dos dados
    name='choropleth',
    data=map_data,
    columns=['NM_MUN', 'avg_d_mbps'],  # Especifica as colunas para o join e para determinar a cor
    key_on='feature.properties.NM_MUN',  # A chave para fazer o join com os dados GeoJSON
    fill_color='YlOrRd',  # Esquema de cores
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Velocidade Média de Internet (Mbps)'
).add_to(m)

# Adiciona um controle de camadas
folium.LayerControl().add_to(m)

m.save('complete_brazil_heatmap.html')
#m.save('sao_paulo_heatmap.html')

# Mostra o mapa
m